In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from contextlib import redirect_stdout
from IPython.core.display import HTML

from fit_incremental import FixedLagFitter
import loader

In [ ]:
traj_data = loader.load_segments('A', index=1)
data = np.vstack(traj_data)[::2]
new_stroke_inds = [0] + np.cumsum([stroke.shape[0] for stroke in traj_data])[:-1] // 2

In [ ]:
plt.plot(data[:, 1], data[:, 2], 'k-')
plt.plot(data[new_stroke_inds, 1], data[new_stroke_inds, 2], 'r*')
plt.axis('equal');

In [ ]:
# fitter = FixedLagFitter(window=1.0)
fitter = FixedLagFitter()
fitter.initialize(data[:10])
actual = [fitter.query(data[:10, 0])]
for i in range(10, data.shape[0]):
    with redirect_stdout(open('/dev/null', 'w')):
        # actual.append(fitter.step(data[i], i in new_stroke_inds))
        actual.append(fitter.step(data[i]))
# actual = np.array(actual)

In [ ]:
fig, ax = plt.subplots()

ax.plot(data[:, 1], data[:, 2], 'k.')
lims = ax.axis('equal')

def f(i):
    est = actual[i]
    ax.clear()
    ax.plot(data[:, 1], data[:, 2], 'k.')
    ax.plot(est[:, 1], est[:, 2], 'r-')
    ax.axis('equal')
    ax.set_xlim(*lims[:2])
    ax.set_ylim(*lims[2:])

anim = FuncAnimation(fig, f, frames=range(len(actual)), interval=100, repeat=False)
HTML(anim.to_jshtml())

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))


def plot_one_speed_profile(ax, i):
    ax.clear()

    i, txy_est, (values, new_stroke_times, strokes, prev_strokes) = fitter.history[i]

    speeds = lambda t: [stroke.speed(t, values=values) for stroke in prev_strokes + strokes]
    all_speeds = np.array([speeds(t) for t in txy_est[:, 0]])

    v = np.diff(txy_est[:, 1:], axis=0) / np.diff(txy_est[:, 0:1], axis=0)
    total_speed = np.sqrt(np.sum(np.square(v), axis=1))

    # print(fitter.previous_strokes)
    # print(speeds(txy_est[0, 0]))
    # print(all_speeds)
    for i, stroke_speed in enumerate(all_speeds.T):
        if i >= len(prev_strokes):
            ax.plot(txy_est[:, 0], stroke_speed, label=f'Stroke {i}', linestyle='--')
        else:
            ax.plot(txy_est[:, 0], stroke_speed, label=f'Stroke {i}')
    ax.plot(txy_est[:-1, 0], total_speed, 'k:', label='Overall Speed')
    xdot_data = np.gradient(data[:, 1:], axis=0) / np.gradient(data[:, 0]).reshape(-1, 1)
    ax.plot(data[:txy_est.shape[0], 0], np.linalg.norm(xdot_data, axis=1)[:txy_est.shape[0]], 'k--', label='Input Data Speed')
    ax.legend(loc='upper left')

# plot_one_speed_profile(10)

def f(i):
    plot_one_speed_profile(ax[0], i)
    _, txy_est, (_, new_stroke_times, _, _) = fitter.history[i]
    ax[1].clear()
    for s, e in zip([0] + new_stroke_inds[:-1].tolist(), new_stroke_inds):
        ax[1].plot(data[s:e, 1], data[s:e, 2], '-', linewidth=3)
    ax[1].plot(data[:, 1], data[:, 2], 'k.')
    ax[1].plot(txy_est[:, 1], txy_est[:, 2], 'r-*')
    # new_stroke = txy_est[:, 0] == new_stroke_times
    # ax[1].plot(txy_est[new_stroke, 1], txy_est[new_stroke, 2], 'r*')
    ax[1].axis('equal')

anim = FuncAnimation(fig,
                     f,
                     frames=range(len(fitter.history)),
                    #  frames=range(5),
                     interval=100)

HTML(anim.to_jshtml())

In [ ]:
anim.save('results/speed_animation_autosegmentation.mp4')

In [ ]:
fig, ax = plt.subplots()
ax.plot(data[:, 1], data[:, 2], 'k-')
txy_online = np.array([est[-1, :] for _, est, _ in fitter.history])
ax.plot(txy_online[:, 1], txy_online[:, 2], 'r*')
ax.axis('equal')

In [ ]:
with open('vid.html', 'w') as f:
    # f.write(anim.to_html5_video())
    f.write(anim.to_jshtml())

In [ ]:
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy.signal
import scipy.integrate


def good_indices(data, threshold=5e-6):
    delta2 = np.sum(np.square(np.diff(data[:, 1:], axis=0)), axis=1)
    delta2_smooth = scipy.signal.savgol_filter(delta2, 11, 1)
    inds = np.abs(delta2) < threshold
    fig, axes = plt.subplots(3, 1)
    tsub = data[1:, 0]
    axes[0].plot(delta2)
    axes[0].plot(delta2_smooth)
    axes[1].plot(data[2:, 0], np.diff(delta2), 'r.')
    axes[1].plot(tsub[inds], np.diff(delta2)[inds[:-1]], 'k.')
    axes[2].plot(np.diff(np.diff(delta2)), '.')
    plt.show()
    
    inds = np.abs(delta2) > -99999
    return np.array(inds.tolist() + [False])

def speed(data, plots=False):
    print(f'{data.shape=}')
    t = data[:, 0]
    # t = scipy.signal.savgol_filter(t, 5, 1)
    # dt = scipy.signal.savgol_filter(np.diff(t), 5, 1)
    dt = np.diff(t, )
    print('********?*?*', dt)
    if plots:
        plt.figure(4)
        plt.subplot(311)
        plt.plot(t[:-1], dt)
        plt.plot(t[:-1], np.diff(t), '--')

        plt.subplot(312)
        plt.plot(t[:-1], np.sqrt(np.sum(np.square(np.diff(data[:, 1:], axis=0)), axis=1)))

    # v = np.diff(data[:, 1:], axis=0) / np.diff(data[:, 0]).reshape(-1, 1)
    # v = np.diff(data[:, 1:], axis=0) / np.diff(t).reshape(-1, 1)
    v = np.diff(data[:, 1:], axis=0) / dt.reshape(-1, 1)
    speed2 = np.sum(np.square(v), axis=1)
    try:
        speed2_smooth = scipy.signal.savgol_filter(speed2, 5, 1)
    except ValueError:
        print('Warning: Trajectory too short to smooth speed for local minima checking: ', speed2)
        speed2_smooth = speed2
    if plots:
        plt.subplot(313)
        plt.plot(t[:-1], speed2)
        # plt.plot(t[:-1], speed2_smooth, )
        plt.show()

    return speed2


def SNR(input_data, output_data):
    numerator = scipy.integrate.simpson(input_data**2)
    denominator = scipy.integrate.simpson((input_data - output_data)**2)
    return 10 * np.log10(numerator / denominator)
    # return 10*np.log(scipy.integrate.simpson(input_data**2) /
    #           scipy.integrate.simpson((input_data-output_data)**2))


##################################################################################

snr_history = fitter.snr_history
full_history = fitter.full_stroke

params = full_history[-1]
print(params)

# history: n rows of [input_stroke, estimated_output], where n is # of points
# input_stroke: np.array(t, x, y) with n(i) rows (n at that iteration)
# estimated_output: np.array(t, x, y)

smooth_acc = snr_history[-1][2]
inflections = snr_history[-1][3]

input_stroke = snr_history[-1][0]
estimated_output = snr_history[-1][1]
output_stroke = []
for _, new_output, _, _ in snr_history:
    output_stroke.extend(new_output[len(output_stroke):])
output_stroke = np.array(output_stroke)

plt.figure(1)
plt.plot(input_stroke[:, 1], input_stroke[:, 2], 'r')
print(input_stroke)
plt.show()
# print(input_stroke.shape, estimated_output.shape, output_stroke.shape)

# print("___________________________________ Input")
# print(input_stroke)
# print("___________________________________ Estimated")
# print(estimated_output)
# print("___________________________________ Output")
# print(output_stroke)

inds = good_indices(input_stroke)
print(f'{inds = }')
input_stroke = input_stroke[inds]
estimated_output = estimated_output[inds]
output_stroke = output_stroke[inds]

in_speed = speed(input_stroke, plots=True)
est_speed = speed(estimated_output)
out_speed = speed(output_stroke)

est_SNR = SNR(in_speed, est_speed)
out_SNR = SNR(in_speed, out_speed)

print(input_stroke)

plt.figure(1)
plt.plot(estimated_output[:-1, 0],
         est_speed,
         'b-',
         linewidth=0.5,
         label='final estimate (SNR: %s)' % float('%.3g' % est_SNR))
plt.plot(output_stroke[:-1, 0],
         out_speed,
         'g-',
         linewidth=0.5,
         label='online output: (SNR: %s)' % float('%.3g' % out_SNR))
plt.plot(input_stroke[:-1, 0], in_speed, 'k-',
         linewidth=2, label='motion input')
# plt.plot(input_stroke[:-1, 0], 1 / np.diff(input_stroke[:, 0]), 'r-', linewidth=1, label='dt output: (SNR: %s)' % float('%.3g' % out_SNR))
plt.ylim(0, 10)
plt.legend()
#output_stroke[:-4, 0], 
print("INFLECTIONS\n",inflections, type(inflections))
# plt.savefig('/tmp/speed.png')
plt.figure(2)
plt.subplot(211)
plt.plot(in_speed)
plt.ylabel('Input Speed')
plt.subplot(212)
plt.plot(smooth_acc, 'g-', linewidth=1, label='smoothed accel')
plt.plot(inflections, smooth_acc[inflections], 'k*', label='inflections')
plt.ylabel('Smoothed (?) Accelerations')
plt.show()
